In [ ]:
from nanome.service import PluginService
from nanome.interface import PluginInstanceRedisInterface

# Set up redis credentials
redis_host = 'redis'
redis_port = 6379
redis_password = ''

# When your PluginService is running, you can get your channel value from the Logs, or from the query parameter in your open browser.
# Update this value to match that, so that your commands will run against your live workspace.
redis_channel = 'a5100bcd-3b33-48dc-a8b0-67f88182f24e'

plugin_instance = PluginInstanceRedisInterface(redis_host, redis_port, redis_password, redis_channel)
print('Hello')

In [ ]:
comps = plugin_instance.request_complex_list()
print(comps)

In [ ]:
# neo4j setup
from py2neo import Graph, Node, Relationship

neo4j_uri = 'bolt://neo4j:7687/'
username = 'neo4j'
password = 'F8dwclytew'
graph = Graph(neo4j_uri, auth=(username, password))


In [ ]:
# Create neo4j database
with open("assets/neo4j_movie_graph_query.txt") as f:
    query_string = f.read()
results = graph.run(query_string)
data = results.data()

In [ ]:
from nanome.api import structure
from nanome.util import Vector3, Color

def create_node_complex(node, atom_position):
    comp = structure.Complex()
    mol = structure.Molecule()
    chain = structure.Chain()
    res = structure.Residue()
    atom = structure.Atom()

    atom.position = atom_position
    atom.labeled = True
    
    label_text = ''
    if node.labels.__str__() == ':Movie':
        # Movie node
        label_text = node['title']
        atom.atom_color = Color(0, 255, 0)
    else:
        # Person Node
        label_text = node['name']
    atom.label_text = label_text
    res.add_atom(atom)
    chain.add_residue(res)
    mol.add_chain(chain)
    comp.add_molecule(mol)
    comp.name = atom.label_text
    return comp

# comp_nodes = []
# position = Vector3(0, 0, 0)
# for record in data:
#     node = record['n']
#     new_comp = create_node_complex(node, position)
#     position = Vector3(position.x + 1, position.y + 1, position.z + 1)
#     comp_nodes.append(new_comp)
# results = plugin_instance.update_structures_deep(comp_nodes)

In [ ]:
import nanome
from nanome.api.shapes import Label, Line, Shape

def draw_relationship_line(struct1, struct2):
    """Draw line connecting two structs.

    :arg struct1: struct
    :arg struct2: struct
    :arg line_settings: Dict describing shape and color of line based on interaction_type
    """
    line = Line()
    for struct, anchor in zip([struct1, struct2], line.anchors):
        anchor.anchor_type = nanome.util.enums.ShapeAnchorType.Atom
        anchor.target = struct.index
    return line
    

In [12]:
result = graph.run("MATCH p=()-[r:ACTED_IN]->() RETURN p LIMIT 25")

position = Vector3(0, 0, 0)
comp_nodes = []
lines = []

for record in result:
    path = record['p']
    nodes = path.nodes
    rels = path.relationships    
    for node in nodes:
        node_uuid = node.__uuid__

        label_text = ''
        if node.labels.__str__() == ':Movie':
            # Movie node
            label_text = node['title']
        else:
            # Person Node
            label_text = node['name']

        if label_text not in [comp.name for comp in comp_nodes]:
            comp = create_node_complex(node, position)
            created_comp = plugin_instance.add_to_workspace([comp])[0]
            comp_nodes.append(created_comp)
            position = Vector3(position.x + 1, position.y + 1, position.z + 1)

#     line = draw_relationship_line(next(path_comp_nodes[0].atoms), next(path_comp_nodes[1].atoms))
#     lines.append(line)
# plugin_instance.upload_shapes(lines)


Sending add_to_workspace Request to Redis Channel a5100bcd-3b33-48dc-a8b0-67f88182f24e
Response received on channel 37c424c6-5213-4676-bb04-29354897869c
Sending add_to_workspace Request to Redis Channel a5100bcd-3b33-48dc-a8b0-67f88182f24e
Response received on channel e763a40a-de17-4139-900c-f8f087288c38


StopIteration: 

In [ ]:
c
